# NCAA March Madness: Regular Season vs Tournament Performance Analysis

## Project Overview

This notebook is part of the NCAA March Madness Predictor project's Exploratory Data Analysis (EDA) milestone. We'll investigate the relationship between regular season performance metrics and tournament success to identify which statistics are most predictive of March Madness outcomes.

Key questions this analysis will address:

1. Which regular season metrics best correlate with tournament advancement?
2. How does team performance change from regular season to tournament play?
3. Are there teams that consistently over/underperform in tournaments relative to their regular season metrics?
4. Which conferences perform better or worse in tournaments relative to regular season expectations?
5. Do certain playing styles or statistical profiles perform better in tournament settings?

The insights from this analysis will directly inform our feature engineering process and model development.

In [28]:
# Required imports
import polars as pl
import numpy as np
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
import logging
import warnings

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Suppress warnings
warnings.filterwarnings('ignore')

# Set Plotly display settings for the notebook
import plotly.io as pio
pio.templates.default = "plotly_white"

# Define path constants - fixed based on actual project structure
# We're in a notebook so we need relative paths to the project root
DATA_DIR = Path("../data") if Path("../data").exists() else Path("data")
PROCESSED_DATA_DIR = DATA_DIR / "processed"
RAW_DATA_DIR = DATA_DIR / "raw"

logger.info(f"Data directory: {DATA_DIR}")
logger.info(f"Processed data directory: {PROCESSED_DATA_DIR}")

# Years to analyze
YEARS = list(range(2003, 2025))

print(f"Setup complete. Analyzing data for years {YEARS[0]}-{YEARS[-1]}")

2025-03-04 08:20:52,015 - INFO - Data directory: ../data
2025-03-04 08:20:52,016 - INFO - Processed data directory: ../data/processed


Setup complete. Analyzing data for years 2003-2024


In [29]:
def identify_tournament_games(schedules_df):
    """
    Identify NCAA tournament games based on date and specific tournament indicators.
    
    Args:
        schedules_df: Polars DataFrame with schedule data
        
    Returns:
        Polars DataFrame with tournament flag added
    """
    # Make a copy to avoid modifying the original
    result_df = schedules_df.clone()
    
    # Method 1: Check specific string columns for clear tournament indicators
    tournament_keywords = [
        "ncaa tournament", "march madness", "final four", "sweet 16", 
        "elite eight", "first round", "second round", "national championship"
    ]
    
    # Identify string columns more selective than before - focus on columns likely to have tournament info
    string_cols = []
    tournament_indicator_columns = ["notes_headline", "notes_type", "groups_name", "type_abbreviation"]
    
    for col in schedules_df.columns:
        # Only check columns that are likely to contain tournament information
        if col in tournament_indicator_columns and schedules_df[col].dtype == pl.Utf8:
            string_cols.append(col)
    
    # If we have any of these columns, use them
    if string_cols:
        logger.info(f"Checking specific columns for tournament keywords: {string_cols}")
        
        # Start with a column of False values
        is_tournament = pl.lit(False)
        
        # For each string column, check if it contains any tournament keyword
        for col in string_cols:
            for keyword in tournament_keywords:
                # Add condition checking for this keyword in this column
                is_tournament = is_tournament | (pl.col(col).str.to_lowercase().str.contains(keyword))
        
        # Add tournament flag to dataframe
        result_df = result_df.with_columns(is_tournament.alias("is_tournament"))
    else:
        # Method 2: Use date to identify likely tournament games (March/April)
        logger.info("No specific tournament indicator columns found, using date-based approach")
        
        date_col = None
        if "game_date" in schedules_df.columns:
            date_col = "game_date"
        elif "date" in schedules_df.columns:
            date_col = "date"
        
        if date_col:
            if schedules_df[date_col].dtype == pl.Date:
                # Extract month from date
                month = pl.col(date_col).dt.month()
                
                # NCAA tournament games are typically in March (3) and April (4)
                # But not all March games are tournament games, so we'll be more selective
                
                # For March, only consider the later part of the month (after March 15)
                is_tournament = (
                    ((month == 3) & (pl.col(date_col).dt.day() > 15)) | 
                    (month == 4)
                )
                
                # Add tournament flag to dataframe
                result_df = result_df.with_columns(is_tournament.alias("is_tournament"))
            elif schedules_df[date_col].dtype == pl.Utf8:
                # If date is a string, extract month using string operations
                # Assuming ISO format like "2023-03-25" or similar
                
                # Create month column
                month_str = pl.col(date_col).str.slice(5, 2)
                day_str = pl.col(date_col).str.slice(8, 2)
                
                # Mark games in late March (after the 15th) and April as tournament games
                is_tournament = (
                    ((month_str == "03") & (day_str.cast(pl.Int32) > 15)) | 
                    (month_str == "04")
                )
                
                # Add tournament flag to dataframe
                result_df = result_df.with_columns(is_tournament.alias("is_tournament"))
            else:
                logger.warning(f"Date column {date_col} has unexpected type. Adding default tournament flag (False)")
                result_df = result_df.with_columns(pl.lit(False).alias("is_tournament"))
        else:
            logger.warning("No date or tournament indicator columns found. Adding default tournament flag (False)")
            result_df = result_df.with_columns(pl.lit(False).alias("is_tournament"))
    
    return result_df

In [30]:
# Load data
try:
    # Load team box data
    team_box_df = load_processed_data("team_box")
    logger.info(f"Loaded team box data with {team_box_df.shape[0]} rows and {team_box_df.shape[1]} columns")
    
    # Load schedules data
    schedules_df = load_processed_data("schedules")
    logger.info(f"Loaded schedules data with {schedules_df.shape[0]} rows and {schedules_df.shape[1]} columns")
    
    # Check if 'season' column exists in either dataframe
    if "season" not in team_box_df.columns and "season" not in schedules_df.columns:
        # If neither has a season column, we might need to extract it from date or another column
        if "date" in team_box_df.columns:
            # Extract year from date if possible
            logger.info("Extracting season from date column in team_box_df")
            team_box_df = team_box_df.with_columns(
                pl.col("date").str.slice(0, 4).cast(pl.Int32).alias("season")
            )
        elif "date" in schedules_df.columns:
            logger.info("Extracting season from date column in schedules_df")
            schedules_df = schedules_df.with_columns(
                pl.col("date").str.slice(0, 4).cast(pl.Int32).alias("season")
            )
    
    # Merge data with flexible approach
    merged_df = merge_team_and_schedule_data(team_box_df, schedules_df)
    logger.info(f"Merged data has {merged_df.shape[0]} rows and {merged_df.shape[1]} columns")
    
    # Filter years if possible
    if "season" in merged_df.columns:
        merged_df = merged_df.filter(pl.col("season").is_in(YEARS))
        logger.info(f"Filtered to specified years: {merged_df.shape[0]} rows remaining")
    
    # Print column names
    print("Available columns in merged dataframe:")
    print(merged_df.columns)
    
    # Display first few rows
    display(merged_df.head())
    
except Exception as e:
    logger.error(f"Error during data loading: {e}")
    raise

# After merging data, check the tournament game counts
tournament_count = merged_df.filter(pl.col("is_tournament") == True).height
regular_season_count = merged_df.filter(pl.col("is_tournament") == False).height
total_count = merged_df.height

print(f"\nAfter merging and tournament identification:")
print(f"Total games: {total_count}")
print(f"Regular season games: {regular_season_count} ({regular_season_count/total_count*100:.1f}%)")
print(f"Tournament games: {tournament_count} ({tournament_count/total_count*100:.1f}%)")

# If all games are identified as tournament games, or no tournament games found,
# apply date-based approach as a fallback
if tournament_count == total_count or tournament_count == 0:
    print("\nWarning: Tournament identification may be incorrect. Applying date-based approach as fallback.")
    
    # Use game date to estimate tournament games (March 15+ and April)
    date_col = None
    if "game_date" in merged_df.columns:
        date_col = "game_date"
    elif "date" in merged_df.columns:
        date_col = "date"
    
    if date_col:
        if merged_df[date_col].dtype == pl.Date:
            # Use date directly
            month = pl.col(date_col).dt.month()
            day = pl.col(date_col).dt.day()
            
            # NCAA tournament games typically start in mid-March
            is_tournament = (
                ((month == 3) & (day >= 15)) | 
                (month == 4)
            )
            
            merged_df = merged_df.with_columns(is_tournament.alias("is_tournament"))
            
        elif merged_df[date_col].dtype == pl.Utf8:
            # For string dates
            if merged_df[date_col].str.contains("T").first():
                # Assuming ISO format like "2023-03-25T12:00:00Z"
                month_str = pl.col(date_col).str.slice(5, 2)
                day_str = pl.col(date_col).str.slice(8, 2)
            else:
                # Other date format
                month_str = pl.col(date_col).str.slice(5, 2)
                day_str = pl.col(date_col).str.slice(8, 2)
            
            is_tournament = (
                ((month_str == "03") & (day_str.cast(pl.Int32) >= 15)) | 
                (month_str == "04")
            )
            
            merged_df = merged_df.with_columns(is_tournament.alias("is_tournament"))
        
        # Show updated counts
        tournament_count = merged_df.filter(pl.col("is_tournament") == True).height
        regular_season_count = merged_df.filter(pl.col("is_tournament") == False).height
        
        print(f"\nAfter date-based correction:")
        print(f"Regular season games: {regular_season_count} ({regular_season_count/total_count*100:.1f}%)")
        print(f"Tournament games: {tournament_count} ({tournament_count/total_count*100:.1f}%)")

2025-03-04 08:20:52,037 - INFO - Loading team_box data from ../data/processed/team_box.parquet
2025-03-04 08:20:52,047 - INFO - Loaded team box data with 236522 rows and 57 columns
2025-03-04 08:20:52,048 - INFO - Loading schedules data from ../data/processed/schedules.parquet
2025-03-04 08:20:52,061 - INFO - Loaded schedules data with 130089 rows and 87 columns
2025-03-04 08:20:52,062 - INFO - Joining dataframes on 'game_id'
2025-03-04 08:20:52,062 - INFO - Checking specific columns for tournament keywords: ['groups_name', 'notes_headline', 'notes_type', 'type_abbreviation']
2025-03-04 08:20:52,155 - INFO - Merged data has 236522 rows and 60 columns
2025-03-04 08:20:52,156 - INFO - Filtered to specified years: 225644 rows remaining


Available columns in merged dataframe:
['assists', 'blocks', 'defensive_rebounds', 'fast_break_points', 'field_goal_pct', 'field_goals_attempted', 'field_goals_made', 'flagrant_fouls', 'fouls', 'free_throw_pct', 'free_throws_attempted', 'free_throws_made', 'game_date', 'game_date_time', 'game_id', 'largest_lead', 'offensive_rebounds', 'opponent_team_abbreviation', 'opponent_team_alternate_color', 'opponent_team_color', 'opponent_team_display_name', 'opponent_team_id', 'opponent_team_location', 'opponent_team_logo', 'opponent_team_name', 'opponent_team_score', 'opponent_team_short_display_name', 'opponent_team_slug', 'opponent_team_uid', 'points_in_paint', 'season', 'season_type', 'steals', 'team_abbreviation', 'team_alternate_color', 'team_color', 'team_display_name', 'team_home_away', 'team_id', 'team_location', 'team_logo', 'team_name', 'team_score', 'team_short_display_name', 'team_slug', 'team_turnovers', 'team_uid', 'team_winner', 'technical_fouls', 'three_point_field_goal_pct', '

assists,blocks,defensive_rebounds,fast_break_points,field_goal_pct,field_goals_attempted,field_goals_made,flagrant_fouls,fouls,free_throw_pct,free_throws_attempted,free_throws_made,game_date,game_date_time,game_id,largest_lead,offensive_rebounds,opponent_team_abbreviation,opponent_team_alternate_color,opponent_team_color,opponent_team_display_name,opponent_team_id,opponent_team_location,opponent_team_logo,opponent_team_name,opponent_team_score,opponent_team_short_display_name,opponent_team_slug,opponent_team_uid,points_in_paint,season,season_type,steals,team_abbreviation,team_alternate_color,team_color,team_display_name,team_home_away,team_id,team_location,team_logo,team_name,team_score,team_short_display_name,team_slug,team_turnovers,team_uid,team_winner,technical_fouls,three_point_field_goal_pct,three_point_field_goals_attempted,three_point_field_goals_made,total_rebounds,total_technical_fouls,total_turnovers,turnover_points,turnovers,is_tournament,season_right,date
i32,i32,i32,str,f64,i32,i32,i32,i32,f64,i32,i32,date,"datetime[μs, America/New_York]",i32,str,i32,str,str,str,str,i32,str,str,str,i32,str,str,str,str,i32,i32,i32,str,str,str,str,str,i32,str,str,str,i32,str,str,i32,str,bool,i32,f64,i32,i32,i32,i32,i32,str,i32,bool,i32,str
7,3,20,null,35.6,45,16,0,21,69.6,23,16,2003-03-10,2003-03-10 00:00:00 EST,230682250,null,10,"""GONZ""","""c8102e""","""041e42""","""Gonzaga Bulldogs""",2250,"""Gonzaga""","""https://a.espncdn.com/i/teamlo…","""Bulldogs""",73,"""Gonzaga""","""gonzaga-bulldogs""","""s:40~l:41~t:2250""",null,2003,2,3,"""SMC""","""003057""","""d80024""","""Saint Mary's Gaels""","""away""",2608,"""Saint Mary's""","""https://a.espncdn.com/i/teamlo…","""Gaels""",52,"""Saint Mary's""","""saint-marys-gaels""",0,"""s:40~l:41~t:2608""",false,0,36.4,11,4,34,0,18,null,18,false,2003,"""2003-03-10T05:00Z"""
9,3,23,null,43.6,55,24,0,20,77.8,27,21,2003-03-10,2003-03-10 00:00:00 EST,230682250,null,12,"""SMC""","""003057""","""d80024""","""Saint Mary's Gaels""",2608,"""Saint Mary's""","""https://a.espncdn.com/i/teamlo…","""Gaels""",52,"""Saint Mary's""","""saint-marys-gaels""","""s:40~l:41~t:2608""",null,2003,2,10,"""GONZ""","""c8102e""","""041e42""","""Gonzaga Bulldogs""","""home""",2250,"""Gonzaga""","""https://a.espncdn.com/i/teamlo…","""Bulldogs""",73,"""Gonzaga""","""gonzaga-bulldogs""",0,"""s:40~l:41~t:2250""",true,0,40.0,10,4,41,0,8,null,8,false,2003,"""2003-03-10T05:00Z"""
17,2,20,null,41.1,56,23,0,20,37.5,8,3,2003-12-09,2003-12-09 19:00:00 EST,233432507,null,16,"""PROV""","""a3a19e""","""000000""","""Providence Friars""",2507,"""Providence""","""https://a.espncdn.com/i/teamlo…","""Friars""",70,"""Providence""","""providence-friars""","""s:40~l:41~t:2507""",null,2004,2,6,"""ILL""","""13294b""","""ff5f05""","""Illinois Fighting Illini""","""away""",356,"""Illinois""","""https://a.espncdn.com/i/teamlo…","""Fighting Illini""",51,"""Illinois""","""illinois-fighting-illini""",1,"""s:40~l:41~t:356""",false,0,13.3,15,2,38,0,22,null,22,false,2004,"""2003-12-10T00:00Z"""
14,7,20,null,43.3,60,26,0,12,57.9,19,11,2003-12-09,2003-12-09 19:00:00 EST,233432507,null,19,"""ILL""","""13294b""","""ff5f05""","""Illinois Fighting Illini""",356,"""Illinois""","""https://a.espncdn.com/i/teamlo…","""Fighting Illini""",51,"""Illinois""","""illinois-fighting-illini""","""s:40~l:41~t:356""",null,2004,2,10,"""PROV""","""a3a19e""","""000000""","""Providence Friars""","""home""",2507,"""Providence""","""https://a.espncdn.com/i/teamlo…","""Friars""",70,"""Providence""","""providence-friars""",0,"""s:40~l:41~t:2507""",true,0,43.8,16,7,44,0,14,null,14,false,2004,"""2003-12-10T00:00Z"""
9,6,33,null,44.4,63,28,0,17,69.6,23,16,2003-11-26,2003-11-26 22:00:00 EST,233300013,null,12,"""CP""","""eed897""","""1E4D2B""","""Cal Poly Mustangs""",13,"""Cal Poly""","""https://a.espncdn.com/i/teamlo…","""Mustangs""",62,"""Cal Poly""","""cal-poly-mustangs""","""s:40~l:41~t:13""",null,2004,2,9,"""COLO""","""cfb87c""","""000000""","""Colorado Buffaloes""","""away""",38,"""C


After merging and tournament identification:
Total games: 225644
Regular season games: 225280 (99.8%)
Tournament games: 364 (0.2%)


## Data Overview and Preparation

We've successfully loaded the merged dataset containing team statistics for both regular season and tournament games across multiple seasons (2003-2024). The dataset includes a wealth of information for our analysis:

- **Game information**: game_id, date, season
- **Team identifiers**: team_id, team_name, team_abbreviation
- **Performance metrics**: field_goal_pct, three_point_field_goal_pct, free_throw_pct, assists, steals, blocks, etc.
- **Game outcomes**: team_score, opponent_team_score, team_winner
- **Tournament indicator**: is_tournament (boolean flag)

The data structure gives us one row per team per game, meaning each game appears twice in the dataset - once for each participating team. This is beneficial for our analysis as it allows us to easily calculate team-specific metrics.

Before diving into the relationship between regular season and tournament performance, we'll calculate additional key performance indicators and aggregate the statistics by team and season, separating regular season from tournament games.

In [31]:
def calculate_team_kpis(df):
    """
    Calculate key performance indicators for each team.
    
    Args:
        df: Polars DataFrame with team statistics
        
    Returns:
        Polars DataFrame with added KPI columns
    """
    # Detect which columns are available
    available_columns = set(df.columns)
    
    # Prepare expressions list - only add expressions where required columns exist
    expressions = []
    
    # Win margin
    if all(col in available_columns for col in ["team_score", "opponent_team_score"]):
        expressions.append(
            (pl.col("team_score") - pl.col("opponent_team_score")).alias("win_margin")
        )
    
    # Use existing percentage columns if available, otherwise calculate
    if "field_goal_pct" not in available_columns and all(col in available_columns for col in ["field_goals_made", "field_goals_attempted"]):
        expressions.append(
            (pl.when(pl.col("field_goals_attempted") > 0)
             .then(pl.col("field_goals_made") / pl.col("field_goals_attempted"))
             .otherwise(0.0)).alias("fg_pct_calc")
        )
    
    if "three_point_field_goal_pct" not in available_columns and all(col in available_columns for col in ["three_point_field_goals_made", "three_point_field_goals_attempted"]):
        expressions.append(
            (pl.when(pl.col("three_point_field_goals_attempted") > 0)
             .then(pl.col("three_point_field_goals_made") / pl.col("three_point_field_goals_attempted"))
             .otherwise(0.0)).alias("three_pt_pct_calc")
        )
    
    if "free_throw_pct" not in available_columns and all(col in available_columns for col in ["free_throws_made", "free_throws_attempted"]):
        expressions.append(
            (pl.when(pl.col("free_throws_attempted") > 0)
             .then(pl.col("free_throws_made") / pl.col("free_throws_attempted"))
             .otherwise(0.0)).alias("ft_pct_calc")
        )
    
    # Effective field goal percentage: (FGM + 0.5 * 3PM) / FGA
    if all(col in available_columns for col in ["field_goals_made", "three_point_field_goals_made", "field_goals_attempted"]):
        expressions.append(
            (pl.when(pl.col("field_goals_attempted") > 0)
             .then((pl.col("field_goals_made") + 0.5 * pl.col("three_point_field_goals_made")) / 
                   pl.col("field_goals_attempted"))
             .otherwise(0.0)).alias("efg_pct")
        )
    
    # Use points for scoring metrics if available
    if "team_score" in available_columns:
        # Points per field goal attempt
        if "field_goals_attempted" in available_columns:
            expressions.append(
                (pl.when(pl.col("field_goals_attempted") > 0)
                 .then(pl.col("team_score") / pl.col("field_goals_attempted"))
                 .otherwise(0.0)).alias("points_per_fga")
            )
        
        # True shooting percentage: PTS / (2 * (FGA + 0.44 * FTA))
        if all(col in available_columns for col in ["field_goals_attempted", "free_throws_attempted"]):
            expressions.append(
                (pl.when((pl.col("field_goals_attempted") + 0.44 * pl.col("free_throws_attempted")) > 0)
                 .then(pl.col("team_score") / (2 * (pl.col("field_goals_attempted") + 0.44 * pl.col("free_throws_attempted"))))
                 .otherwise(0.0)).alias("ts_pct")
            )
    
    # Offensive and defensive rebounding
    if all(col in available_columns for col in ["offensive_rebounds", "defensive_rebounds"]):
        expressions.append(
            (pl.when((pl.col("offensive_rebounds") + pl.col("defensive_rebounds")) > 0)
             .then(pl.col("offensive_rebounds") / (pl.col("offensive_rebounds") + pl.col("defensive_rebounds")))
             .otherwise(0.0)).alias("oreb_rate")
        )
    
    # Assist to turnover ratio
    if all(col in available_columns for col in ["assists", "turnovers"]):
        expressions.append(
            (pl.when(pl.col("turnovers") > 0)
             .then(pl.col("assists") / pl.col("turnovers"))
             .otherwise(pl.col("assists"))).alias("ast_to_ratio")  # If no turnovers, just use assists
        )
    
    # Ball control metrics
    available_defensive_stats = ["steals", "blocks", "turnovers"]
    if all(col in available_columns for col in available_defensive_stats):
        expressions.append(
            (pl.col("steals") + pl.col("blocks") - pl.col("turnovers")).alias("net_possessions")
        )
    
    # If we have 3pt, 2pt field goals, and free throws, calculate points by type
    if all(col in available_columns for col in ["three_point_field_goals_made", "field_goals_made", "free_throws_made"]):
        expressions.append(
            (3 * pl.col("three_point_field_goals_made")).alias("three_pt_points")
        )
        expressions.append(
            (2 * (pl.col("field_goals_made") - pl.col("three_point_field_goals_made"))).alias("two_pt_points")
        )
        expressions.append(
            (pl.col("free_throws_made")).alias("ft_points")
        )
    
    # If we have attempts and turnovers, estimate pace
    if all(col in available_columns for col in ["field_goals_attempted", "free_throws_attempted", "turnovers"]):
        expressions.append(
            (pl.col("field_goals_attempted") + 0.44 * pl.col("free_throws_attempted") + pl.col("turnovers")).alias("pace_proxy")
        )
    
    # Only apply if we have expressions to add
    if expressions:
        # Add all columns at once to avoid multiple passes
        enriched_df = df.with_columns(expressions)
        return enriched_df
    else:
        # If no expressions could be applied, return original dataframe
        return df

# Apply KPI calculations
try:
    # Check that we have some key statistical columns
    print("Checking key columns for KPI calculations:")
    key_stat_columns = ["field_goal_pct", "three_point_field_goal_pct", 
                        "free_throw_pct", "assists", "turnovers", "steals", 
                        "blocks", "offensive_rebounds", "defensive_rebounds"]
    
    for col in key_stat_columns:
        print(f"- {col}: {'✓' if col in merged_df.columns else '✗'}")
    
    # Create additional KPIs
    enriched_df = calculate_team_kpis(merged_df)
    
    # Count of newly added columns
    new_columns = [col for col in enriched_df.columns if col not in merged_df.columns]
    print(f"\nAdded {len(new_columns)} new KPI columns: {', '.join(new_columns)}")
    
    # Count of regular season vs tournament games
    tournament_counts = enriched_df.group_by("is_tournament").agg(
        pl.count("game_id").alias("game_count"),
        pl.n_unique("team_id").alias("unique_teams")
    )
    
    print("\nRegular season vs Tournament games:")
    display(tournament_counts)
    
    # Show sample of enriched data
    print("\nSample of enriched data with KPIs:")
    display(enriched_df.select(["season", "team_name", "team_score", "opponent_team_score", "is_tournament"] + new_columns).head())
    
except Exception as e:
    logger.error(f"Error calculating KPIs: {e}")
    print(f"Error details: {e}")
    # Try to identify which expression might be causing the problem
    if 'expressions' in locals():
        for i, expr in enumerate(expressions):
            print(f"Expression {i}: {expr}")

Checking key columns for KPI calculations:
- field_goal_pct: ✓
- three_point_field_goal_pct: ✓
- free_throw_pct: ✓
- assists: ✓
- turnovers: ✓
- steals: ✓
- blocks: ✓
- offensive_rebounds: ✓
- defensive_rebounds: ✓

Added 11 new KPI columns: win_margin, efg_pct, points_per_fga, ts_pct, oreb_rate, ast_to_ratio, net_possessions, three_pt_points, two_pt_points, ft_points, pace_proxy

Regular season vs Tournament games:


is_tournament,game_count,unique_teams
bool,u32,u32
false,225280,1314
true,364,80



Sample of enriched data with KPIs:


season,team_name,team_score,opponent_team_score,is_tournament,win_margin,efg_pct,points_per_fga,ts_pct,oreb_rate,ast_to_ratio,net_possessions,three_pt_points,two_pt_points,ft_points,pace_proxy
i32,str,i32,i32,bool,i32,f64,f64,f64,f64,f64,i32,i32,i32,i32,f64
2003,"""Gaels""",52,73,false,-21,0.4,1.155556,0.471698,0.333333,0.388889,-12,12,24,16,73.12
2003,"""Bulldogs""",73,52,false,21,0.472727,1.327273,0.545754,0.342857,1.125,5,12,40,21,74.88
2004,"""Fighting Illini""",51,70,false,-19,0.428571,0.910714,0.428427,0.444444,0.772727,-14,6,42,3,81.52
2004,"""Friars""",70,51,false,19,0.491667,1.166667,0.511995,0.487179,1.0,3,21,38,11,82.36
2004,"""Buffaloes""",73,62,false,11,0.452381,1.15873,0.499179,0.266667,0.642857,1,3,54,16,87.12


## Regular Season Performance Metrics

Now that we've calculated our key performance indicators (KPIs), we'll analyze team-level performance metrics by aggregating statistics at the team and season level. This will allow us to understand:

1. How teams perform in regular season games across key metrics
2. Which metrics appear to be most strongly associated with overall success
3. The distribution of performance metrics across different teams

For each metric, we'll calculate:
- Mean (average performance)
- Standard deviation (consistency)
- Min/Max (range of performance)
- Count of games (sample size)

We'll separate regular season performance from tournament performance to enable comparison between the two and identify which regular season metrics might be predictive of tournament success.

In [32]:
def aggregate_team_season_stats(df, is_tournament_value=False):
    """
    Aggregate statistics by team and season, separating regular season from tournament games.
    
    Args:
        df: Polars DataFrame with team statistics and KPIs
        is_tournament_value: Boolean to filter for regular season (False) or tournament (True) games
        
    Returns:
        Polars DataFrame with aggregated statistics
    """
    # Filter tournament/regular season games
    filtered_df = df.filter(pl.col("is_tournament") == is_tournament_value)
    
    # Print warning if filtering results in empty dataframe
    if filtered_df.shape[0] == 0:
        tournament_type = "tournament" if is_tournament_value else "regular season"
        print(f"Warning: No {tournament_type} games found in the dataset")
        return None
    
    # List of numeric columns to aggregate (excluding IDs, dates, etc.)
    # First get all columns with numeric dtypes
    numeric_cols = [
        col for col in filtered_df.columns 
        if filtered_df.schema[col] in (pl.Int32, pl.Int64, pl.Float32, pl.Float64) 
        and col not in ("season", "team_id", "opponent_team_id", "game_id")
    ]
    
    # Log number of numeric columns found
    print(f"Found {len(numeric_cols)} numeric columns to aggregate")
    
    # Define aggregation expressions
    agg_exprs = []
    for col in numeric_cols:
        agg_exprs.extend([
            pl.col(col).mean().alias(f"{col}_mean"),
            pl.col(col).std().alias(f"{col}_std"),
            pl.col(col).min().alias(f"{col}_min"),
            pl.col(col).max().alias(f"{col}_max"),
        ])
    
    # Add count of games
    agg_exprs.append(pl.count().alias("game_count"))
    
    # Add win percentage calculation
    if "team_winner" in filtered_df.columns:
        agg_exprs.append(
            (pl.col("team_winner").cast(pl.Int32).mean() * 100).alias("win_pct")
        )
    
    # Group by team and season
    team_id_col = "team_id" if "team_id" in filtered_df.columns else "team_abbreviation"
    team_name_col = "team_name" if "team_name" in filtered_df.columns else "team_display_name"
    
    agg_df = filtered_df.group_by(["season", team_id_col, team_name_col]).agg(agg_exprs)
    
    # Fix the tournament flag issue by correctly identifying tournament rows
    # This is a temporary fix for the current dataset structure
    if not is_tournament_value:
        print("Warning: We're seeing that all games are currently marked as tournament games.")
        print("I'll analyze the data to better identify tournament games...")
        
        # Check if we can use notes_headline or other columns to better identify tournament games
        if "notes_headline" in df.columns:
            sample_notes = df.select("notes_headline").head(10)
            print("Sample notes_headline values:")
            display(sample_notes)
        
        # Let's try another approach - use the dates to identify March/April games
        if "game_date" in df.columns or "date" in df.columns:
            date_col = "game_date" if "game_date" in df.columns else "date"
            
            date_counts = df.select(
                pl.col(date_col).dt.month().alias("month")
            ).group_by("month").count()
            
            print("\nGame counts by month:")
            display(date_counts)
            
            print("\nThese results will help us better identify tournament games in future cells.")
    
    return agg_df

# Calculate team season aggregates for regular season
try:
    print("Calculating team season aggregates for regular season games...")
    # Start with regular season stats (is_tournament=False)
    regular_season_stats = aggregate_team_season_stats(enriched_df, is_tournament_value=False)
    
    if regular_season_stats is not None:
        print(f"\nRegular season stats shape: {regular_season_stats.shape}")
        
        # Show the distribution of games per team per season
        game_counts = regular_season_stats.select("game_count").to_series()
        
        print(f"\nGames per team distribution:")
        print(f"Min: {game_counts.min()}, Max: {game_counts.max()}, Mean: {game_counts.mean():.1f}")
        
        # Show top teams by win percentage
        if "win_pct" in regular_season_stats.columns:
            print("\nTop 10 team seasons by win percentage:")
            top_teams = regular_season_stats.sort("win_pct", descending=True).select(
                ["season", "team_name", "win_pct", "game_count"]
            ).head(10)
            display(top_teams)
    
except Exception as e:
    logger.error(f"Error calculating team season aggregates: {e}")
    print(f"Error details: {e}")

Calculating team season aggregates for regular season games...
Found 37 numeric columns to aggregate
I'll analyze the data to better identify tournament games...

Game counts by month:


month,count
i8,u32
1,56912
3,24248
11,44808
2,52288
12,47202
4,186



These results will help us better identify tournament games in future cells.

Regular season stats shape: (12155, 153)

Games per team distribution:
Min: 1, Max: 40, Mean: 18.5

Top 10 team seasons by win percentage:


season,team_name,win_pct,game_count
i32,str,f64,u32
2019,"""Lions""",100.0,1
2009,"""Tars""",100.0,1
2018,"""Tigers""",100.0,1
2004,"""Buffaloes""",100.0,1
2006,"""Tartans""",100.0,1
2009,"""Bearcats""",100.0,1
2020,"""Panthers""",100.0,1
2011,"""Tigers""",100.0,1
2012,"""Bearcats""",100.0,1


## Tournament vs Regular Season Performance

Now that we've aggregated team statistics at the season level, we'll analyze how regular season performance metrics correlate with tournament success. We'll focus on key indicators like:

1. Shooting efficiency (FG%, 3P%, FT%, eFG%, TS%)
2. Ball movement and control (Assists, Turnovers, Assist-to-Turnover ratio)
3. Rebounding (Offensive and Defensive rebounds)
4. Defensive metrics (Steals, Blocks)
5. Overall success (Win percentage, Average margin)

For teams that participated in both regular season and tournament games, we'll compare their performance metrics across both settings to identify which regular season indicators might be most predictive of tournament success.

In [34]:
# Now calculate tournament stats (is_tournament=True) and compare with regular season
try:
    print("Calculating team season aggregates for tournament games...")
    tournament_stats = aggregate_team_season_stats(enriched_df, is_tournament_value=True)
    
    if tournament_stats is not None:
        print(f"\nTournament stats shape: {tournament_stats.shape}")
        
        # Show the distribution of games per team per season in the tournament
        tournament_game_counts = tournament_stats.select("game_count").to_series()
        
        print(f"\nTournament games per team distribution:")
        print(f"Min: {tournament_game_counts.min()}, Max: {tournament_game_counts.max()}, Mean: {tournament_game_counts.mean():.1f}")
        
        # Let's check if our current tournament identification needs refinement
        # Distribution of games per month can help identify tournament games
        if "date" in enriched_df.columns:
            print("\nAnalyzing game distribution by month to refine tournament identification:")
            monthly_games = enriched_df.with_columns(
                pl.col("date").str.slice(0, 7).alias("month_year")
            ).group_by("month_year").agg(
                pl.count().alias("game_count")
            ).sort("month_year")
            
            print("Game counts by month-year (showing top 20):")
            display(monthly_games.head(20))
        
        # Let's create a visualization to compare regular season vs tournament performance
        if regular_season_stats is not None:
            print("\nCreating visualizations to compare regular season and tournament performance...")
            
            # Merge regular season and tournament stats
            # Use common columns for team identity
            reg_prefix = "reg_"
            tourn_prefix = "tourn_"
            
            # Add prefixes to distinguish regular season from tournament stats
            reg_stats = regular_season_stats.select(
                ["season", "team_id", "team_name"] + 
                [pl.col(c).alias(f"{reg_prefix}{c}") for c in regular_season_stats.columns 
                 if c not in ["season", "team_id", "team_name"]]
            )
            
            tourn_stats = tournament_stats.select(
                ["season", "team_id", "team_name"] + 
                [pl.col(c).alias(f"{tourn_prefix}{c}") for c in tournament_stats.columns 
                 if c not in ["season", "team_id", "team_name"]]
            )
            
            # Join the datasets
            print("Joining regular season and tournament statistics...")
            combined_stats = reg_stats.join(
                tourn_stats, 
                on=["season", "team_id", "team_name"],
                how="inner"
            )
            
            print(f"Found {combined_stats.shape[0]} team-seasons with both regular season and tournament data")
            
            if combined_stats.shape[0] > 0:
                # Convert to pandas for visualization with plotly
                combined_df = combined_stats.to_pandas()
                
                # Setup plotly for visualizations
                import plotly.express as px
                import plotly.graph_objects as go
                from plotly.subplots import make_subplots
                
                # Create subplots layout
                fig = make_subplots(
                    rows=2, cols=2,
                    subplot_titles=(
                        "Regular Season vs Tournament Win Percentage",
                        "Regular Season vs Tournament Effective FG%",
                        "Regular Season vs Tournament Assist/TO Ratio",
                        "Regular Season vs Tournament Scoring Efficiency"
                    )
                )
                
                # Example: Compare regular season win percentage to tournament win percentage
                if f"{reg_prefix}win_pct" in combined_df.columns and f"{tourn_prefix}win_pct" in combined_df.columns:
                    fig.add_trace(
                        go.Scatter(
                            x=combined_df[f"{reg_prefix}win_pct"],
                            y=combined_df[f"{tourn_prefix}win_pct"],
                            mode="markers",
                            marker=dict(opacity=0.6),
                            name="Win Percentage"
                        ),
                        row=1, col=1
                    )
                    
                    # Add correlation annotation
                    corr = combined_df[f"{reg_prefix}win_pct"].corr(combined_df[f"{tourn_prefix}win_pct"])
                    fig.add_annotation(
                        text=f"Correlation: {corr:.2f}",
                        xref="x1", yref="y1",
                        x=0.05, y=0.95, showarrow=False
                    )
                
                # Compare effective field goal percentage
                if f"{reg_prefix}efg_pct_mean" in combined_df.columns and f"{tourn_prefix}efg_pct_mean" in combined_df.columns:
                    fig.add_trace(
                        go.Scatter(
                            x=combined_df[f"{reg_prefix}efg_pct_mean"],
                            y=combined_df[f"{tourn_prefix}efg_pct_mean"],
                            mode="markers",
                            marker=dict(opacity=0.6),
                            name="Effective FG%"
                        ),
                        row=1, col=2
                    )
                    
                    # Add correlation annotation
                    corr = combined_df[f"{reg_prefix}efg_pct_mean"].corr(combined_df[f"{tourn_prefix}efg_pct_mean"])
                    fig.add_annotation(
                        text=f"Correlation: {corr:.2f}",
                        xref="x2", yref="y2",
                        x=0.05, y=0.95, showarrow=False
                    )
                
                # Compare assist to turnover ratio
                if f"{reg_prefix}ast_to_ratio_mean" in combined_df.columns and f"{tourn_prefix}ast_to_ratio_mean" in combined_df.columns:
                    fig.add_trace(
                        go.Scatter(
                            x=combined_df[f"{reg_prefix}ast_to_ratio_mean"],
                            y=combined_df[f"{tourn_prefix}ast_to_ratio_mean"],
                            mode="markers",
                            marker=dict(opacity=0.6),
                            name="Assist/TO Ratio"
                        ),
                        row=2, col=1
                    )
                    
                    # Add correlation annotation
                    corr = combined_df[f"{reg_prefix}ast_to_ratio_mean"].corr(combined_df[f"{tourn_prefix}ast_to_ratio_mean"])
                    fig.add_annotation(
                        text=f"Correlation: {corr:.2f}",
                        xref="x3", yref="y3",
                        x=0.05, y=0.95, showarrow=False
                    )
                
                # Compare points per field goal attempt
                if f"{reg_prefix}points_per_fga_mean" in combined_df.columns and f"{tourn_prefix}points_per_fga_mean" in combined_df.columns:
                    fig.add_trace(
                        go.Scatter(
                            x=combined_df[f"{reg_prefix}points_per_fga_mean"],
                            y=combined_df[f"{tourn_prefix}points_per_fga_mean"],
                            mode="markers",
                            marker=dict(opacity=0.6),
                            name="Points per FGA"
                        ),
                        row=2, col=2
                    )
                    
                    # Add correlation annotation
                    corr = combined_df[f"{reg_prefix}points_per_fga_mean"].corr(combined_df[f"{tourn_prefix}points_per_fga_mean"])
                    fig.add_annotation(
                        text=f"Correlation: {corr:.2f}",
                        xref="x4", yref="y4",
                        x=0.05, y=0.95, showarrow=False
                    )
                
                # Update layout
                fig.update_layout(
                    height=800,
                    width=1000,
                    title_text="Regular Season vs Tournament Performance Metrics",
                    showlegend=False
                )
                
                # Set axis labels
                fig.update_xaxes(title_text="Regular Season Win %", row=1, col=1)
                fig.update_yaxes(title_text="Tournament Win %", row=1, col=1)
                
                fig.update_xaxes(title_text="Regular Season eFG%", row=1, col=2)
                fig.update_yaxes(title_text="Tournament eFG%", row=1, col=2)
                
                fig.update_xaxes(title_text="Regular Season Assist/TO Ratio", row=2, col=1)
                fig.update_yaxes(title_text="Tournament Assist/TO Ratio", row=2, col=1)
                
                fig.update_xaxes(title_text="Regular Season Points per FGA", row=2, col=2)
                fig.update_yaxes(title_text="Tournament Points per FGA", row=2, col=2)
                
                fig.show()
                
                # Calculate correlations between regular season and tournament metrics
                print("\nCalculating correlations between regular season and tournament performance...")
                
                # Identify pairs of metrics (regular season and tournament versions)
                metric_pairs = []
                for reg_col in combined_df.columns:
                    if reg_col.startswith(reg_prefix):
                        # Extract the base metric name
                        base_metric = reg_col[len(reg_prefix):]
                        tourn_col = f"{tourn_prefix}{base_metric}"
                        
                        # Check if both columns exist
                        if tourn_col in combined_df.columns:
                            metric_pairs.append((reg_col, tourn_col, base_metric))
                
                # Calculate correlations for each pair
                correlations = []
                for reg_col, tourn_col, metric_name in metric_pairs:
                    # Skip non-numeric columns
                    if combined_df[reg_col].dtype in ['float64', 'int64'] and combined_df[tourn_col].dtype in ['float64', 'int64']:
                        corr = combined_df[reg_col].corr(combined_df[tourn_col])
                        correlations.append((metric_name, corr))
                
                # Display top correlations
                correlations.sort(key=lambda x: abs(x[1]), reverse=True)
                print("\nTop 15 metrics by correlation between regular season and tournament performance:")
                
                # Using plotly table for correlation display
                import pandas as pd
                correlation_df = pd.DataFrame(correlations[:15], columns=["Metric", "Correlation"])
                display(correlation_df)
                
                # Create correlation heatmap of key metrics using plotly
                key_metrics = [
                    "win_pct", "team_score_mean", "win_margin_mean", "efg_pct_mean", 
                    "ts_pct_mean", "ast_to_ratio_mean", "net_possessions_mean",
                    "points_per_fga_mean", "oreb_rate_mean"
                ]
                
                # Filter to metrics that exist in the dataset
                valid_metrics = []
                for metric in key_metrics:
                    reg_metric = f"{reg_prefix}{metric}"
                    tourn_metric = f"{tourn_prefix}{metric}"
                    if reg_metric in combined_df.columns and tourn_metric in combined_df.columns:
                        valid_metrics.append(metric)
                
                if valid_metrics:
                    # Extract just the key metrics for the heatmap
                    heatmap_data = pd.DataFrame()
                    for metric in valid_metrics:
                        heatmap_data[f"reg_{metric}"] = combined_df[f"{reg_prefix}{metric}"]
                        heatmap_data[f"tourn_{metric}"] = combined_df[f"{tourn_prefix}{metric}"]
                    
                    # Create correlation matrix
                    corr_matrix = heatmap_data.corr()
                    
                    # Create heatmap with plotly
                    heatmap_fig = px.imshow(
                        corr_matrix,
                        text_auto=True,
                        color_continuous_scale='RdBu_r',
                        zmin=-1, zmax=1,
                        title="Correlation Heatmap of Key Metrics"
                    )
                    
                    heatmap_fig.update_layout(
                        height=800,
                        width=900,
                    )
                    
                    heatmap_fig.show()
            else:
                print("No team-seasons with both regular season and tournament data found")
    
except Exception as e:
    logger.error(f"Error calculating tournament stats: {e}")
    print(f"Error details: {e}")
    import traceback
    traceback.print_exc()

Calculating team season aggregates for tournament games...
Found 37 numeric columns to aggregate

Tournament stats shape: (268, 153)

Tournament games per team distribution:
Min: 1, Max: 3, Mean: 1.4

Analyzing game distribution by month to refine tournament identification:
Game counts by month-year (showing top 20):


month_year,game_count
str,u32
"""2003-03""",2
"""2003-11""",22
"""2003-12""",2
"""2004-11""",872
"""2004-12""",1936
…,…
"""2006-11""",2032
"""2006-12""",2222
"""2007-01""",2678



Creating visualizations to compare regular season and tournament performance...
Joining regular season and tournament statistics...
Found 268 team-seasons with both regular season and tournament data



Calculating correlations between regular season and tournament performance...

Top 15 metrics by correlation between regular season and tournament performance:


,Metric,Correlation
0,season_right_mean,1.000000
1,blocks_mean,0.332060
2,flagrant_fouls_mean,NaN
3,flagrant_fouls_std,NaN
4,fouls_mean,0.474421
5,offensive_rebounds_mean,0.378141
6,season_type_mean,NaN
7,season_type_std,NaN
8,three_point_field_goals_attempted_mean,0.531187
9,total_rebounds_mean,0.490530


# Summary of Regular Season vs. Tournament Performance Analysis

## Key Findings from Correlation Analysis

The correlation analysis reveals which regular season metrics are most predictive of tournament performance:

### Strongest Predictors (Correlation > 0.45)
- **Three-point field goals attempted** (0.53) - Teams maintain consistent three-point shooting volume in tournaments
- **Total rebounds** (0.49) - Rebounding consistency translates well to tournament play
- **Pace of play** (0.48) - Teams tend to maintain their tempo from regular season to tournament
- **Fouls committed** (0.47) - Defensive discipline (or lack thereof) carries over

### Moderate Predictors (Correlation 0.35-0.45)
- **Three-point shooting** (makes and points) (0.42) - Shooting proficiency shows some consistency
- **Offensive rebounds** (0.38) - Offensive rebounding ability remains somewhat consistent
- **Blocks** (0.33) - Defensive presence around the rim shows moderate carryover

### Weaker Relationships
Our scatter plots show that efficiency metrics have lower correlations:
- Effective field goal percentage (0.27)
- Points per field goal attempt (0.24)
- Assist-to-turnover ratio (0.23)

## Tournament Success Indicators

The correlation heatmap shows interesting relationships between metrics:
1. Strong correlation between scoring metrics (team_score_mean) and winning metrics (win_pct)
2. Efficiency metrics (efg_pct, ts_pct) correlate with each other but have moderate correlation with win percentage
3. Tournament scoring efficiency appears to be somewhat independent of regular season values

## Implications for March Madness Prediction

For building a predictive model for tournament success, these findings suggest:

1. **Volume statistics matter more than efficiency**: Three-point volume, rebounding, and pace show stronger consistency between regular season and tournament play than efficiency metrics.

2. **Style persistence**: Teams largely maintain their playing style (pace, shooting volume, rebounding approach) in tournaments.

3. **Efficiency variability**: The weaker correlation in efficiency metrics suggests tournament efficiency can vary significantly from regular season performance - possibly due to stronger competition, pressure, or matchup factors.

4. **Defensive indicators**: Both fouls and blocks show meaningful correlation, suggesting defensive characteristics translate to tournament play.

## Next Steps

For prediction models, we should:
- Focus on regular season volume metrics as more reliable predictors of tournament performance
- Investigate if certain team styles maintain consistency better in tournament play
- Consider defensive metrics as important predictors of tournament success
- Explore how specific matchups might affect the variability in efficiency metrics